In [2]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import keras
from keras import backend as K

In [8]:
from unet_model import unet_mdense

In [10]:
TF_ENABLE_ONEDNN_OPTS=0

In [2]:
BACKBONE='resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [11]:
def jacard_coef(y_true, y_pred):
    # y_true=y_true.astype(np.uint8)
    # y_prede=y_pred.astype(np.uint8)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0))


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

In [16]:
def binary_tangent(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tangent=tf.tan((y_true_f-y_pred_f))
    return (K.sum(tangent)/float(len(tangent)))

def binary_tangent_loss(y_true, y_pred):
    return -binary_tangent(y_true, y_pred)

In [5]:
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.5, 0.5])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [5]:
SIZE_X = 256
SIZE_Y = 256

In [20]:
train_images = []
train_labels=[]
for directory_path in (glob.glob("SBU-shadow/SBUTrain4KRecoveredSmall/ShadowImages")):
    for img_path in sorted(glob.glob(os.path.join(directory_path, "*.jpg"))):
#         print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

NameError: name 'label' is not defined

In [ ]:
train_masks = [] 
for directory_path in sorted(glob.glob("SBU-shadow/SBUTrain4KRecoveredSmall/ShadowMasks/")):
    for mask_path in sorted(glob.glob(os.path.join(directory_path, "*.png"))):
        print(mask_path)
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        train_labels.append(label)
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [6]:
X=np.load('imgs.npy')
Y=np.load('masks.npy')
Y = np.expand_dims(Y, axis=3)
Y=(Y>0.5).astype(np.float32)

In [15]:
# X=train_images
# Y=train_masks
# Y = np.expand_dims(Y, axis=3)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

NameError: name 'preprocess_input' is not defined

In [12]:
model=unet_mdense(256,256,3,1,0.1)
model.compile(optimizer= 'adam', loss=[jacard_coef_loss
                                      ],
              metrics = [jacard_coef])
print(model.summary())

NameError: name 'jacard_coef_lossa' is not defined

In [19]:
history=model.fit(x_train, 
          y_train,
          batch_size=16, 
          epochs=10,
          verbose=1,
          validation_data=(x_val, y_val))

NameError: name 'X_train' is not defined

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_nuclei.h5', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]

history=model.fit(x_train, 
                  y_train,
                  batch_size=16, 
                  epochs=10,
                  callbacks=callbacks,
                  verbose=1,
                  validation_data=(x_val, y_val))

Epoch 1/10
  4/205 [..............................] - ETA: 3:17 - loss: -0.2667 - binary_tangent: 0.2667